In [1]:
!pip install upstash_redis langchain langchain_redis langchain_community langchain-groq --q

In [2]:
from google.colab import userdata
import os
GROQ_API_KEY = userdata.get('GROQ_API_KEY').strip()
UPSTASH_REDIS_REST_TOKEN = userdata.get('UPSTASH_REDIS')
UPSTASH_REDIS_REST_URL=userdata.get('UPSTASH_REDIS_REST_URL')
os.environ["GROQ_API_KEY"]=userdata.get('GROQ_API_KEY').strip()

In [ ]:
from langchain_community.chat_message_histories import UpstashRedisChatMessageHistory


def get_redis_history(session_id:int):
  return UpstashRedisChatMessageHistory(
    url=UPSTASH_REDIS_REST_URL,
    token=UPSTASH_REDIS_REST_TOKEN,
    session_id=session_id,
)

In [ ]:
history=UpstashRedisChatMessageHistory(
    url=UPSTASH_REDIS_REST_URL,
    token=UPSTASH_REDIS_REST_TOKEN,
    session_id='session_id',
)


In [3]:
from langchain_core.prompts import ChatPromptTemplate ,MessagesPlaceholder
from langchain_groq import ChatGroq
from pydantic import BaseModel, Field, validator
from typing import List

llm = ChatGroq(
    model='openai/gpt-oss-120b',
    temperature=0.3,
    reasoning_format="parsed",
    max_retries=2,
)


In [ ]:
!pip install --upgrade torch torchvision torchaudio transformers --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 107.2 MB/s eta 0:00:00


In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

response_schemas = [
    ResponseSchema(name="section", description="Workout section: Warm-up, Main Workout, or Cool-down"),
    ResponseSchema(name="exercise", description="Name of the exercise"),
    ResponseSchema(name="sets", description="Number of sets (integer or null if not applicable)"),
    ResponseSchema(name="reps", description="Number of reps per set (integer or null if not applicable)"),
    ResponseSchema(name="duration", description="Duration or time spent on this exercise, e.g., '3 minutes'"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"section": string  // Workout section: Warm-up, Main Workout, or Cool-down
	"exercise": string  // Name of the exercise
	"sets": string  // Number of sets (integer or null if not applicable)
	"reps": string  // Number of reps per set (integer or null if not applicable)
	"duration": string  // Duration or time spent on this exercise, e.g., '3 minutes'
}
```


In [4]:
fitness_prompt = ChatPromptTemplate.from_messages([
    ("system", f"""
You are an expert world-class fitness coach and motivational trainer.
Design personalized daily workouts that are:
1. Safe and goal-oriented
2. Fit user’s equipment and time
3. Progressively challenging over 4 weeks
4. Motivational, engaging, and varied

Follow structure:
- Warm-up (mobility/cardio)
- Main workout (mission-specific)
- Cool-down (stretching/recovery)

Incorporate previous session context from chat history to ensure progression and recovery balance.

You must output valid JSON following this schema:
{{format_instructions}}

Return ONLY the JSON — no explanations or extra text.
"""),

MessagesPlaceholder(variable_name="history"),

    ("human","{input}")
])


In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

USER_SESSION_ID = "test_user_42"
user_inputs = {
    "day": 1,
    "week": 1,
    "day_of_week": "Monday",
    "mission": "Build lean muscle and increase deadlift 1RM.",
    "time_commitment": "45 minutes",
    "gear": "Dumbbells (medium/heavy), resistance bands",
    "squad": "The Iron Crew (focus on strength and intensity)"
}
user_prompt=f"""
Generate a workout for **Day {user_inputs['day']} (Week {user_inputs['week']}, {user_inputs['day_of_week']})**.

User Profile:
- Goal: {user_inputs['mission']}
- Time: {user_inputs['time_commitment']}
- Gear: {user_inputs["gear"]}
- Squad Type: {user_inputs['squad']}

Constraints:
1. Fit within time limit
2. Use only listed gear
3. Focus on goal area
4. Include warm-up, main workout, and cool-down
5. Be specific with sets/reps/duration
6. Adjust intensity by week
7. Reference previous plan for balance

Return strictly as JSON (array of exercise objects).
"""
chain = fitness_prompt | llm

workout_chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_redis_history,
    input_messages_key="input",
    history_messages_key="history"
)

USER_SESSION_ID = "test_user_42"

print(f"Generating workout for session: {USER_SESSION_ID}...")

result = workout_chain_with_history.invoke(
    {"input":user_prompt,'format_instructions':format_instructions},
    config={"configurable": {"session_id": USER_SESSION_ID}}
)

print(result)


Generating workout for session: test_user_42...
content='```json\n[\n    {\n        "section": "Warm-up",\n        "exercise": "Jump Rope",\n        "sets": null,\n        "reps": null,\n        "duration": "5 minutes"\n    },\n    {\n        "section": "Warm-up",\n        "exercise": "Dynamic Mobility (leg swings, hip circles, band dislocates, shoulder pass-throughs)",\n        "sets": null,\n        "reps": null,\n        "duration": "5 minutes"\n    },\n    {\n        "section": "Main Workout",\n        "exercise": "Dumbbell Romanian Deadlift",\n        "sets": "3",\n        "reps": "8",\n        "duration": "5 minutes"\n    },\n    {\n        "section": "Main Workout",\n        "exercise": "Goblet Squat",\n        "sets": "3",\n        "reps": "10",\n        "duration": "6 minutes"\n    },\n    {\n        "section": "Main Workout",\n        "exercise": "Single-Arm Dumbbell Row (each side)",\n        "sets": "3",\n        "reps": "8",\n        "duration": "6 minutes"\n    },\n    {\

In [ ]:
result.pretty_print()

================================== Ai Message ==================================

```json
[
    {
        "section": "Warm-up",
        "exercise": "Jump Rope",
        "sets": null,
        "reps": null,
        "duration": "5 minutes"
    },
    {
        "section": "Warm-up",
        "exercise": "Dynamic Mobility (leg swings, hip circles, band dislocates, shoulder pass-throughs)",
        "sets": null,
        "reps": null,
        "duration": "5 minutes"
    },
    {
        "section": "Main Workout",
        "exercise": "Dumbbell Romanian Deadlift",
        "sets": "3",
        "reps": "8",
        "duration": "5 minutes"
    },
    {
        "section": "Main Workout",
        "exercise": "Goblet Squat",
        "sets": "3",
        "reps": "10",
        "duration": "6 minutes"
    },
    {
        "section": "Main Workout",
        "exercise": "Single-Arm Dumbbell Row (each side)",
        "sets": "3",
        "reps": "8",
        "duration": "6 minutes"
    },
    {
        "sec

In [13]:
from langchain_core.output_parsers import StrOutputParser
def genarate_motivation(user_inputs):
  prompt = f"""Generate a short, powerful motivational message (max 2 sentences) for day {user_inputs['day']} of a workout plan.

User Profile:
- Mission: {user_inputs['mission']}
- Squad: {user_inputs['squad']}
- Time: {user_inputs['time_commitment']}

The message should:
- Be inspiring and energetic
- Reference their squad identity ({user_inputs['squad']})
- Connect to their mission ({user_inputs['mission']})
- Be unique and not repetitive

Return ONLY the motivational message, nothing else."""
  format_instuction="""
Return ONLY the motivational message, nothing else.
"""
  motivation_chain= fitness_prompt | llm | StrOutputParser()
  motivation_messaage=motivation_chain.invoke({"input":prompt,'format_instructions':format_instuction ,'history':[]})

  return motivation_messaage

In [14]:
user_inputs = {
    "day": 1,
    "mission": "Build lean muscle and increase deadlift 1RM.",
    "time_commitment": "45 minutes",
    "gear": "Dumbbells (medium/heavy), resistance bands",
    "squad": "The Iron Crew (focus on strength and intensity)"
}

result = genarate_motivation(user_inputs)
print(result)

{
  "motivational_message": "Rise, Iron Crew—today we forge lean muscle and crank that deadlift 1RM higher than ever! Let every rep roar with strength and intensity."
}
